# [LVV-T231] - M13T-004: Individual Hardpoint Breakaway Test

Notebook containing data analysis for the [LVV-T231] test case.  
The script used to run this test case can be found in [M13T004.py].  

[LVV-T231]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T231
[M13T004.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T004.py

## Summary

1. The analysis comprining the measured hardpoint stiffness from specified stiffness.
2. This notebook analyses the offline data produces by the script [M13T004.py] and stored on m1m3-dev.cp.lsst.org.
3. The data show the forces changing with hardpoint extension and compression.
4. Compression is marked bya "-" infront of "-999999999"in the file name.

## Expected Results

It is expected that the fitted hardpoint stiffness is within the specified stiffness. 



## Prepare notebook

In [ ]:
## Elevation = 20 deg
# t_start = "2023-05-30T08:26"
# sal_index = 100047

## Elevation = 90 deg
t_start = "2023-05-31T05:40"
sal_index = 100061

# Number of hard points in M1M3
number_of_hardpoints = 6

# scale factor for "encoder"
scale = 0.2442

# number of points around zero force for linear fit
n_points = 100

# Specified stiffnest from the requirements 
spect_stiffness  = 100

# Kernel size for edge detection 
kernel_size = 100

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import os

from astropy.time import Time, TimeDelta
from scipy.signal import find_peaks
from lsst.sitcom import vandv

## Query the data

In [ ]:
t_start = Time(t_start, format="isot", scale="utc")
t_end = t_start + TimeDelta(24 * 3600, format="sec")

client = vandv.efd.create_efd_client()

In [ ]:
df_script = await client.select_time_series(
    "lsst.sal.Script.logevent_logMessage", 
    ["salIndex", "level", "message"], 
    t_start,
    t_end, 
)

# Select messages with the associated SAL Index
df_script = df_script[df_script["salIndex"] == sal_index]

In [ ]:
df_hp = await client.select_time_series(
    "lsst.sal.MTM1M3.hardpointActuatorData", 
    "*", 
    Time(df_script.index[0]), # Tests start
    Time(df_script.index[-1]), # Tests end
)

## Analyze the data

Let's start analyzing the data by having a look at the measured forces over time.  
This will help identifying any potential issues with one of the hard points right away. 

In [ ]:
for hp_index in range(number_of_hardpoints):
    
    df_script_sub = df_script[df_script["message"].str.contains(f"Hard point {hp_index + 1} test")]
    df = df_hp.loc[df_script_sub.index[0]:df_script_sub.index[-1]]
    hp_test = df_hp_test[df_hp_test["hardpointActuator"] == hp_index + 1]
    
    # Create plot axes
    fig, ax = plt.subplots(num=f"hp_timeline_{hp_index}", figsize=(10, 3), dpi=96)
    
    # Show the data
    ax.plot(df[f"measuredForce{hp_index}"])
    
    # Add some props to the plot
    ax.grid("-", lw=0.5, alpha=0.25)
    ax.set_ylabel(f"HP{hp_index + 1} Measured\n Forces [N]")
    ax.set_xlabel("Time [UTC]")
    
    # Format x-axis with time
    ax.xaxis.set_major_locator(mdates.MinuteLocator())  
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  
    
    # Clean up the plot
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    fig.suptitle(f"Individual Hard Points Breakaway Test\n"
                 f"HP{hp_index+1} - SAL Index {sal_index} - {df.index[0].strftime('%Y-%m-%d')}")

    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"plots/m1m3004_sal{sal_index}_hp{hp_index+1}_timeline.png", dpi=150, bbox_inches="tight")
    
    fig.tight_layout()
    fig.autofmt_xdate()
    plt.show()

### Slope split using edge detection

In [ ]:
# Create plot axes
hp_index = 0
height=1000

fig, ax = plt.subplots(num=f"slope_split_{hp_index}", figsize=(10, 3), dpi=96)

col_force = f"measuredForce{hp_index}"
col_encoder = f"encoder{hp_index}"

df_script_sub = df_script[df_script["message"].str.contains(f"Hard point {hp_index + 1} test")]
df = df_hp.loc[df_script_sub.index[0]:df_script_sub.index[-1]]

edge_kernel = (
    np.concatenate(
        [1 * np.ones(int(kernel_size / 2)), -1 * np.ones(int(kernel_size / 2))]
    ) / kernel_size
)
edge = np.convolve(df[col_force], edge_kernel, mode="same")
ax.plot(edge)

# times = df.index
# starts = times[find_peaks(edge, height=height)[0]]
# stops = times[find_peaks(-1 * edge, height=height)[0]]

# # Create plot axes
# fig, ax = plt.subplots(num=f"hp_timeline_{hp_index}", figsize=(10, 3), dpi=96)

# # Show the data
# ax.plot(df[f"measuredForce{hp_index}"])

# # Add some props to the plot
# ax.grid("-", lw=0.5, alpha=0.25)
# ax.set_ylabel(f"HP{hp_index + 1} Measured\n Forces [N]")
# ax.set_xlabel("Time [UTC]")

# # Format x-axis with time
# ax.xaxis.set_major_locator(mdates.MinuteLocator())  
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  

# # Clean up the plot
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

Valid hard point data collection has positive and negative measured forces.  
They should have the same value on both sides.  
Peaks do not count. We want to have plateaus in both sides. 

In [ ]:
for hp_index in range(number_of_hardpoints):
    
    col_force = f"measuredForce{hp_index}"
    col_encoder = f"encoder{hp_index}"
    
    df_script_sub = df_script[df_script["message"].str.contains(f"Hard point {hp_index + 1} test")]
    df = df_hp.loc[df_script_sub.index[0]:df_script_sub.index[-1]]
    df = df[[col_force, col_encoder]]
    
    # Convert encoder into steps (or vice-versa?)
    # df[col_encoder] *= scale
    
    # Extract x and y data
    x = df[col_encoder]
    y = df[col_force]
    
    # Initial plot with all the data
    fig, ax = plt.subplots(num=f"HP{hp_index}", figsize=(13, 5), dpi=96)
    ax.plot(df[col_encoder], df[col_force])
    
    # Valid hard point breakaway tests have data crossing 0 N
    if df[col_force].abs().min() < 10:

        # Sort the absolute values of the forces and get the n_points smallest ones
        zero_force_idxs = df[col_force].abs().sort_values().index[:n_points]
    
        # Extract x and y data around zero force
        x_near_zero = x.loc[zero_force_idxs]
        y_near_zero = y.loc[zero_force_idxs]
    
        # Calculate slope of linear fit to data around zero force
        m, b = np.polyfit(x_near_zero, y_near_zero, 1)
        
        # Repeat for evaluate the polynomial 
        zero_force_idxs_eval = df[col_force].abs().sort_values().index[:2*n_points]
    
        x_plot_mean = x.loc[zero_force_idxs_eval]
        x_plot_spec = x.loc[zero_force_idxs_eval]
        y_plot_spec = y.loc[zero_force_idxs_eval]        
    
        ax.plot(
            x_plot_mean, 
            m * x_plot_mean + b, 
            color='red', 
            linewidth=2, 
            label=f"Calculated Stiffness = {m:.2f} N/μm \n"
                  f"Encoder Fitting Range =  {round(x_near_zero.min(),2)}"
                  f" - {round(x_near_zero.max(),2)} μm"
        )
    
        # ax.plot(
        #     x_plot_spec, 
        #     spect_stiffness * x_plot_spec, 
        #     color='green', 
        #     linewidth=2, 
        #     label=f"Spect. Stiffness = 100 N/μm"
        # )
    
        legend = ax.legend(loc='lower right', fontsize=12)
        
        # Modify x-axis to have zero in the middle
        ax.spines['bottom'].set_position('zero')

    # Add axis labels
    ax.set_xlabel('Encoder (steps)')
    ax.set_ylabel('Force (N)')

    # Add grid and legend to plot
    ax.grid("-", lw=0.5, alpha=0.25)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')
    ax.set_ylim(1.1 * y.min(), 1.1 * y.max())
    ax.set_xlim(x_plot_mean.mean() - 2000, x_plot_mean.mean() + 2000)

    # Add title to plot
    title = (f"Individual Hard Point Breakaway Test\n"
             f"HP{hp_index + 1} - SAL Index {sal_index} -"
             f" {df.index[0].isoformat(timespec='seconds')[:-6]} -"
             f" {df.index[-1].isoformat(timespec='seconds')[:-6]}")
    plt.title(f"{title}")

    # Save plot as png file
    filename = f"m1m3t004_hp{hp_index+1}_salidx{sal_index}_steps.png"
    os.makedirs("plots", exist_ok=True)
    fig.savefig(f"plots/{filename}", dpi=150, bbox_inches="tight")

    # Show plot
    plt.show()